In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import networkx as nx

In [35]:
from collections import defaultdict

# Load the datasets

In [4]:
df_nodes = pd.read_csv("datasets/en-wikipedia.humans.ungraph.names.txt", comment="#", delimiter="\t", names=["id", "name"])

In [5]:
df_edges = pd.read_csv("datasets/en-wikipedia.humans.ungraph.txt", comment="#", delimiter="\t", names=["id", "src", "dst"])

In [6]:
df_features = pd.read_csv("datasets/en-wikipedia.humans.ungraph.features.txt", comment="#", delimiter="\t", names=["id", "features"])

df_features["features"] = df_features.features.str.split(" ")

In [7]:
df_features_names = pd.read_csv("datasets/en-wikipedia.humans.ungraph.features.names.txt", comment="#", delimiter="\t", names=["id", "name"])
df_features_names.head()

,id,name
0,0,United States
1,1,Association football
2,2,England
3,3,Germany
4,4,World War II


In [8]:
df_lists_names = pd.read_csv("datasets/en-wikipedia.humans.lists.names", comment="#", delimiter="\t", names=["id", "name", "human_proportion"])

In [9]:
df_lists = pd.read_csv("datasets/en-wikipedia.humans.lists", comment="#", delimiter="\t", names=["id", "list"])

In [10]:
list_presidents_id = map(int, df_lists.iloc[10662]["list"].split())

In [11]:
list_presidents_vect = df_features[df_features["id"].isin(list_presidents_id)]
list_losers_vect = df_features[~df_features["id"].isin(list_presidents_id)]

In [12]:
import networkx as nx

In [13]:
list_presidents_id = map(str, list_presidents_id)

In [14]:
graph = nx.read_edgelist("datasets/en-wikipedia.humans.ungraph.txt", comments="#", delimiter="\t")


In [15]:
csr_graph = nx.adj_matrix(graph)

In [16]:
type(csr_graph)

scipy.sparse.csr.csr_matrix

In [17]:
csr_graph.shape

(731293, 731293)

In [18]:
n_nodes = csr_graph.shape[0]

In [19]:
nodename2nodeid = {node: i for i, node in enumerate(graph.nodes())}

In [20]:
print df_nodes[df_nodes["name"]=="Bill Clinton"]
print df_nodes[df_nodes["name"]=="Abraham Lincoln"]

           id          name
52338  238779  Bill Clinton
            id             name
202217  848214  Abraham Lincoln


In [80]:
seed_set = set(["238779", "848214"])
seed_set = set([id for id in list_presidents_id[: len(list_presidents_id)/ 2]])

In [81]:
n_iter = 10
alpha = 0.85
threshold = 1e-6

ppr = {node: 1. / float(len(seed_set)) for node in seed_set}
for i in range(n_iter):
    new_ppr = defaultdict(float)
    for node in ppr:
        new_ppr[node] += (1. - alpha) * ppr[node]
        node_degree = graph.degree(node)
        for neighbor in graph.neighbors(node):
            new_ppr[neighbor] += (1. / float(node_degree)) * ppr[node]
    ppr = dict()
    for node in new_ppr:
        if new_ppr[node] > threshold:
            ppr[node] = new_ppr[node]

In [87]:
sorted_ppr = sorted(ppr, key=ppr.get, reverse=True)
print(sorted_ppr[:10])
print(len(np.intersect1d(list_presidents_id, sorted_ppr[:63])))
for i in range(20):
    print(df_nodes[df_nodes["id"]==int(sorted_ppr[i])]["name"])
    


[u'238701', u'848214', u'238779', u'239617', u'238700', u'239609', u'277299', u'281645', u'239610', u'241453']
44
52301    George W. Bush
Name: name, dtype: object
202217    Abraham Lincoln
Name: name, dtype: object
52338    Bill Clinton
Name: name, dtype: object
52857    Ronald Reagan
Name: name, dtype: object
52300    Barack Obama
Name: name, dtype: object
52849    Richard Nixon
Name: name, dtype: object
64711    Franklin D. Roosevelt
Name: name, dtype: object
67698    Andrew Jackson
Name: name, dtype: object
52850    Lyndon B. Johnson
Name: name, dtype: object
53486    Dwight D. Eisenhower
Name: name, dtype: object
52852    John F. Kennedy
Name: name, dtype: object
201438    Ulysses S. Grant
Name: name, dtype: object
52855    Jimmy Carter
Name: name, dtype: object
52856    George H. W. Bush
Name: name, dtype: object
983357    George Washington
Name: name, dtype: object
52916    Harry S. Truman
Name: name, dtype: object
71596    Thomas Jefferson
Name: name, dtype: object
201442    Da

In [91]:
a = [1, 2, 3]
b = ["1", "2", "3", "4"]
np.isin(b, a)

array([False, False, False, False], dtype=bool)

In [92]:
a= np.array([1, 2, 3, 4])

In [94]:
mask = np.array([1, 3])
a[mask]

array([2, 4])